fastText indilir ve oluşturulur


In [52]:
!git clone https://github.com/facebookresearch/fastText.git

fatal: destination path 'fastText' already exists and is not an empty directory.


In [53]:
!pip install ./fastText/

Processing ./fastText
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2829893 sha256=0dc357bbd0fd4cff9cc74573fbbe7aabcf69e0ab04a29473a0a2122b392a3bd1
  Stored in directory: /tmp/pip-ephem-wheel-cache-foadx5rg/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext
  Found existing installation: fasttext 0.9.1
    Uninstalling fasttext-0.9.1:
      Successfully uninstalled fasttext-0.9.1


Classification'da kullanılacak olan verisetimizi alıyoruz.

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
PREPROCESSED_DATASET_WITH_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_with_stemming.csv"
PREPROCESSED_DATASET_WITHOUT_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_no_stemming.csv"
PREPROCESSED_DATASET_ZEMBEREK = "gdrive/My Drive/mbti/preprocessed_dataset_zemberek"
TRIMMED_DATASET = "gdrive/My Drive/mbti/trimmed_dataset.csv"
RAW_DATASET = "gdrive/My Drive/mbti/all_users_v2.csv"

In [0]:
import pandas as pd 
df = pd.read_csv(PREPROCESSED_DATASET_WITHOUT_STEMMER, sep = ';', header = 0)

In [77]:
df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,19991991,ekşi itiraf döndüm dolaştım geldim kendime çat...,ENTJ,analysts,E,N,T,J
1,19991991,selda bağcan sesi dinleyebilmek büyük mutluluk...,ENTJ,analysts,E,N,T,J
2,19991991,eski sevgilisinin mutlu olmasını isteyen insan...,ENTJ,analysts,E,N,T,J
3,19991991,veda ederken not bırakmak farklı acılar mutlul...,ENTJ,analysts,E,N,T,J
4,19991991,ingiliz aksanı aralar bayıldığım konuşan insan...,ENTJ,analysts,E,N,T,J
...,...,...,...,...,...,...,...,...
524835,zaimoglu,zlatan ibrahimoviç türkiye katil,ESFJ,sentinels,E,S,F,J
524836,zaimoglu,tarihteki büyük yalancılar şike operasyonu azi...,ESFJ,sentinels,E,S,F,J
524837,zaimoglu,akp chp koalisyonu hayali koalisyondur,ESFJ,sentinels,E,S,F,J
524838,zaimoglu,trabzonlu insana hamsi diyen fenerbahçe taraft...,ESFJ,sentinels,E,S,F,J


Olmayan değerler silinir

In [0]:
df = df.dropna().reset_index(drop = True)

In [79]:
df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,19991991,ekşi itiraf döndüm dolaştım geldim kendime çat...,ENTJ,analysts,E,N,T,J
1,19991991,selda bağcan sesi dinleyebilmek büyük mutluluk...,ENTJ,analysts,E,N,T,J
2,19991991,eski sevgilisinin mutlu olmasını isteyen insan...,ENTJ,analysts,E,N,T,J
3,19991991,veda ederken not bırakmak farklı acılar mutlul...,ENTJ,analysts,E,N,T,J
4,19991991,ingiliz aksanı aralar bayıldığım konuşan insan...,ENTJ,analysts,E,N,T,J
...,...,...,...,...,...,...,...,...
524788,zaimoglu,zlatan ibrahimoviç türkiye katil,ESFJ,sentinels,E,S,F,J
524789,zaimoglu,tarihteki büyük yalancılar şike operasyonu azi...,ESFJ,sentinels,E,S,F,J
524790,zaimoglu,akp chp koalisyonu hayali koalisyondur,ESFJ,sentinels,E,S,F,J
524791,zaimoglu,trabzonlu insana hamsi diyen fenerbahçe taraft...,ESFJ,sentinels,E,S,F,J


fastText classification yaparken labellı verinin ``` __label__LabelValue ``` şeklinde olmasını bekliyor. Burada ilgili dataframe'i bu hale getirilir

In [80]:
df['J/P'] = ["__label__" + class_label for class_label in df['J/P']]

df

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,19991991,ekşi itiraf döndüm dolaştım geldim kendime çat...,ENTJ,analysts,E,N,T,__label__J
1,19991991,selda bağcan sesi dinleyebilmek büyük mutluluk...,ENTJ,analysts,E,N,T,__label__J
2,19991991,eski sevgilisinin mutlu olmasını isteyen insan...,ENTJ,analysts,E,N,T,__label__J
3,19991991,veda ederken not bırakmak farklı acılar mutlul...,ENTJ,analysts,E,N,T,__label__J
4,19991991,ingiliz aksanı aralar bayıldığım konuşan insan...,ENTJ,analysts,E,N,T,__label__J
...,...,...,...,...,...,...,...,...
524788,zaimoglu,zlatan ibrahimoviç türkiye katil,ESFJ,sentinels,E,S,F,__label__J
524789,zaimoglu,tarihteki büyük yalancılar şike operasyonu azi...,ESFJ,sentinels,E,S,F,__label__J
524790,zaimoglu,akp chp koalisyonu hayali koalisyondur,ESFJ,sentinels,E,S,F,__label__J
524791,zaimoglu,trabzonlu insana hamsi diyen fenerbahçe taraft...,ESFJ,sentinels,E,S,F,__label__J


Veriseti %80 train %20 test olacak şekilde ikiye ayrılır. Train veriseti ile ilgili model oluşturulacaktır.

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, shuffle = True, random_state = 42)


In [82]:
train

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
518586,warherooguzhan,bursa çekilen varlık fotoğrafı uzaylı sandım a...,ISTJ,sentinels,I,S,T,__label__J
178419,varos cloudstrider,irish stew sindidun kurulmuş sırbistan kökenli...,ENTJ,analysts,E,N,T,__label__J
93404,macenas,ekim barış pınarı harekatının başlaması savaş ...,INTJ,analysts,I,N,T,__label__J
156624,stormblade,euro eteğin araba paspasına benzetilmesi dünya...,INTJ,analysts,I,N,T,__label__J
156296,teletabiii,adobe after effects kişi fikir vermiş yazılımı...,ENTP,analysts,E,N,T,__label__P
...,...,...,...,...,...,...,...,...
110268,okunmuspenaa,servispoint msi firmanın otistikleri yetkili s...,ENTJ,analysts,E,N,T,__label__J
259178,kareem said,mountain view bugun barack obama bulunmasi cok...,ENFP,diplomats,E,N,F,__label__P
365838,tuzbuzz,konserle seyahati seven kitap okuyan kavgacı e...,ENFP,diplomats,E,N,F,__label__P
131932,patlak fermuar,vechain reddit kulislerinde dönen dedikoduya o...,ENTJ,analysts,E,N,T,__label__J


In [83]:
test

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
190857,2k,whatsapp tan sevgililerin iletişimi kelimeyi t...,ENFJ,diplomats,E,N,F,__label__J
149243,sanal gezgin,olumlu sonraki hayatımın mottosu olacak sözcük...,INTP,analysts,I,N,T,__label__P
385353,asabi,facebook sohbet zımbırtısı pencere açılmıyor,ISFP,explorers,I,S,F,__label__P
440582,biberanides,atom bombası atılınca gerekenler pantolonun pa...,ESTJ,sentinels,E,S,T,__label__J
159448,shed,evi getirmek uğraşma hayali ev parasına arsayı...,ENTP,analysts,E,N,T,__label__P
...,...,...,...,...,...,...,...,...
385265,asabi,galatasaray şike davasından doğal konuşulan al...,ISFP,explorers,I,S,F,__label__P
309165,n8,nigel jong basina oynamaya calisan oyuncu aldi...,ENFJ,diplomats,E,N,F,__label__J
493113,seveni olmayan adam,mayıs ekrem imamoğlu esnafla diyaloğu ekrem ba...,ESTJ,sentinels,E,S,T,__label__J
209250,birdyfirdy,kasım alanya yaşanan motosiklet kazası haberle...,ENFP,diplomats,E,N,F,__label__P


Train ve test olan verisetlerimizin bir dosyaya kaydedilip ilgili pathlerinin fastText'e geçilmesi gerekiyor. O yüzden bu adımda ilgili verisetleri dosyalara kaydedilir. quoting = csv.QUOTE_NONE kaydedilen string değerlerin herhangi bir quote (",') karakteriyle sarılmamasını söyler

In [0]:
import csv

In [0]:
TRAIN_DATASET_PATH = "gdrive/My Drive/mbti/fastText_J-P_train_set.txt"
train.to_csv(TRAIN_DATASET_PATH, header = False, index = False, columns = ['J/P', 'entry'], sep = ' ', quoting = csv.QUOTE_NONE, escapechar = ' ')

In [0]:
TEST_DATASET_PATH = "gdrive/My Drive/mbti/fastText_J-P_test_set.txt"
test.to_csv(TEST_DATASET_PATH, header = False, index = False, columns = ['J/P', 'entry'], sep = ' ', quoting = csv.QUOTE_NONE, escapechar=' ')

FastText'i kullanılabilmek için ilgili kütüphaneyi import ediyoruz.

In [0]:
import fasttext

Modelimizi varsayılan parametrelerle eğitiyoruz

In [0]:
mbti_model = fasttext.train_supervised(TRAIN_DATASET_PATH)

Modeldeki label'lar gösterilir

In [89]:
mbti_model.labels

['__label__P', '__label__J']

Oluşturulan modeli kaydediyoruz.

In [0]:
mbti_model.save_model('gdrive/My Drive/mbti/fastText_J-P_model.bin')

Sonuçları tutmak için gerekli olan dictionary oluşturulur.

In [0]:
results = {
    'predicted': {
        'J': {'actual': {'J': 0, 'P': 0}},
        'P': {'actual': {'J': 0, 'P': 0}}
    }

}

In [0]:
test = test.reset_index(drop = True)

In [93]:
test

,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,2k,whatsapp tan sevgililerin iletişimi kelimeyi t...,ENFJ,diplomats,E,N,F,__label__J
1,sanal gezgin,olumlu sonraki hayatımın mottosu olacak sözcük...,INTP,analysts,I,N,T,__label__P
2,asabi,facebook sohbet zımbırtısı pencere açılmıyor,ISFP,explorers,I,S,F,__label__P
3,biberanides,atom bombası atılınca gerekenler pantolonun pa...,ESTJ,sentinels,E,S,T,__label__J
4,shed,evi getirmek uğraşma hayali ev parasına arsayı...,ENTP,analysts,E,N,T,__label__P
...,...,...,...,...,...,...,...,...
104954,asabi,galatasaray şike davasından doğal konuşulan al...,ISFP,explorers,I,S,F,__label__P
104955,n8,nigel jong basina oynamaya calisan oyuncu aldi...,ENFJ,diplomats,E,N,F,__label__J
104956,seveni olmayan adam,mayıs ekrem imamoğlu esnafla diyaloğu ekrem ba...,ESTJ,sentinels,E,S,T,__label__J
104957,birdyfirdy,kasım alanya yaşanan motosiklet kazası haberle...,ENFP,diplomats,E,N,F,__label__P


In [0]:
for i in range(test.shape[0]):
  y_actual = str(test['J/P'][i]).replace('__label__', '')

  test_entry = test['entry'][i]
  y_predicted = mbti_model.predict(test_entry, k = 1)[0][0]
  y_predicted = y_predicted.replace('__label__', '')

  results['predicted'][y_predicted]['actual'][y_actual] += 1

In [95]:
results

{'predicted': {'J': {'actual': {'J': 29413, 'P': 21075}},
  'P': {'actual': {'J': 21891, 'P': 32580}}}}